In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm


# --- Configuration ---
base_folder = os.path.join("/media", "ssudhakar", "DATA_10TB", "data_solver", "standingwave")
base_path = Path(base_folder)
scheme_map = {'1': 'RK1', '2': 'RK2', '3': 'RK3', '4': 'RK4', '5': 'AB2', '6': 'AB2RK2'}
grid_map = {'1': 128, '2': 512, '3': 1024}
precision_map = {'2': '64bit', '3': '128bit'}

# --- Load All Data into Dictionary ---
data = {}
folders = sorted(base_path.glob('case1_*'))

for folder in folders:
    folder_name = folder.name
    xyz = folder_name.split('_')[1]

    scheme = scheme_map[xyz[0]]
    grid_size = grid_map[xyz[1]]
    precision = precision_map.get(xyz[2], None)
    if precision is None:
        continue

    key = f"{scheme}_G{grid_size}_P{precision}"
    colnames = ["time"] + [f"col{col_num:04d}" for col_num in range(1, grid_size + 2)]

    try:
        var_dict = {}
        for var in tqdm([ "dens", "pres", "temp", "u",  # General physical variables
                        "h2", "h", "o", "o2", "oh", "h2o", "ho2", "h2o2", "ar", "n2"]):  # Chemical species
            fname = folder / f"output_sol_{var}_{xyz}.csv"
            df = pd.read_csv(fname, header=None, names=colnames, dtype=str).astype(np.longdouble)
            var_dict[var] = df
        data[key] = var_dict
    except FileNotFoundError:
        print(f"Missing file in {folder}")


mkey = {}

for s in ['1', '2', '3', '4', '5', '6']:  # schemes
    scheme = scheme_map[s]
    for g in ['1', '2', '3']:  # grid sizes
        grid_size = grid_map[g]
        for p in ['2', '3']:  # precisions
            precision = precision_map[p].replace('-', '')  
            xyz = f"{s}{g}{p}"
            full_key = f"{scheme}_G{grid_size}_P{precision}"
            mkey[xyz] = full_key

svar1 = [ "dens", "pres", "temp", "u",  # General physical variables
          "h2", "h", "o", "o2", "oh", "h2o", "ho2", "h2o2", "ar", "n2"]  # Chemical species

grid_128_case1 = np.linspace(0, 1, 128, endpoint=False)
grid_512_case1 = np.linspace(0, 1, 512, endpoint=False)
grid_1024_case1 = np.linspace(0, 1, 1024, endpoint=False)

stride512 = 4
stride1024 = 8


# x1 = '4'
# z1 = '3'
# svar_num = 3
for svar_num in range(len(svar1)):  # variables 
    for x1 in tqdm(['1', '2', '3', '4', '5', '6']):  # schemes
        for z1 in ['2', '3']:  # precisions
            y1 = 1

            time = data[mkey[f"{x1}{y1+0}{z1}"]][svar1[svar_num]].values[:, 0]
            var1 = data[mkey[f"{x1}{y1+0}{z1}"]][svar1[svar_num]].values[:, 1:-1]
            var2 = data[mkey[f"{x1}{y1+1}{z1}"]][svar1[svar_num]].values[:, 1:-1]
            var3 = data[mkey[f"{x1}{y1+2}{z1}"]][svar1[svar_num]].values[:, 1:-1]
            varref = data[mkey["433"]][svar1[svar_num]].values[:, 1:-1]

            diff1 = np.abs(np.abs(varref[:, ::stride1024]) - np.abs(var1[:, :]))
            diff2 = np.abs(np.abs(varref[:, ::stride1024]) - np.abs(var2[:, ::stride512]))
            diff3 = np.abs(np.abs(varref[:, ::stride1024]) - np.abs(var3[:, ::stride1024]))

            fig, ax = plt.subplots(figsize=(10,3))
            for t in range(0, 200, 5):
                ax.plot(grid_128_case1, diff1[t, :], color='blue', label='$Nx_{128}$')
                ax.plot(grid_128_case1, diff2[t, :], color='green', ls='--', label='$Nx_{512}$')
                ax.plot(grid_128_case1, diff3[t, :], color='red', ls='--', label='$Nx_{1024}$')

            ax.ticklabel_format(axis='y', style='sci', useMathText=True)
            ax.set_title(f"{scheme_map[x1]} Scheme for timestepping\n{precision_map[z1]}-bit precision")
            ax.set_xlabel("Domain")
            ax.set_ylabel('Abs. Error = $| \\varepsilon |$')
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = dict(zip(labels, handles))
            plt.legend(by_label.values(), by_label.keys())
            plt.grid(visible=True, which='both', color='gray', lw=0.5, ls='--', alpha=0.5, zorder=0)
            plt.tight_layout(pad=0.05)
            plt.savefig(os.path.join(base_folder, "images_stats_max_diff", f"stdwave_max_error_{x1}_{z1}_{svar1[svar_num]}.png"), dpi=600)
            plt.close('all')


            with open(os.path.join(base_folder, f"stats_max_error_{svar1[svar_num]}.txt"), mode='a') as stdwave_stats_file:
                stdwave_stats_file.write(f"The reference scheme used is RK4 scheme - 128 bit precision - 1024 grid points\n")
                stdwave_stats_file.write(f"Max difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{y1+0}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff1):.12f}\n")
                stdwave_stats_file.write(f"Max difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{y1+1}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff2):.12f}\n")
                stdwave_stats_file.write(f"Max difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{y1+2}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff3):.12f}\n")
                stdwave_stats_file.write("*******\n*******")
                stdwave_stats_file.write(f"Avg. difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{y1+0}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmean(diff1):.12f}\n")
                stdwave_stats_file.write(f"Avg. difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{y1+1}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmean(diff2):.12f}\n")
                stdwave_stats_file.write(f"Avg. difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{y1+2}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmean(diff3):.12f}\n")
                stdwave_stats_file.write("*******\n*******\n\n\n")

                # print(f"The reference scheme used is RK4 scheme - 128 bit precision - 1024 grid points")
                # print(f"Max difference between ref. and {scheme_map[x1]} - {grid_map[f'{y1}']} grid points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff1):.12f}")
                # print(f"Max difference between ref. and {scheme_map[x1]} - {grid_map[f'{y1+1}']} grid points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff2):.12f}")
                # print(f"Max difference between ref. and {scheme_map[x1]} - {grid_map[f'{y1+2}']} grid points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff3):.12f}")